In [ ]:
import pandas as pd
import numpy as np 
import pickle
from feature_extractor import FeatureExtractor
import os
import requests
from tqdm import tqdm
import pickle
import json
from time import sleep


BASE_URL = "https://benchmarking-scoring.aixplain.io/"
headers = {
  'Content-Type': 'application/json'
}

os.environ["CUDA_VISIBLE_DEVICES"]="1"

def clean_target(text):
    return text.replace("<v>", "").replace("</v>", "")

def get_scores(source, target):
    input_payload = {
            "input": f'["{source}"]',
            "output": f'["{target}"]',
            "refs": "",
            "mode": "string",
            "data_uri": "string",
            "metric_name": "string"
            }
    tmp_response = requests.request("POST", BASE_URL+"startasynctranslationscores", headers=headers, data=json.dumps(input_payload))
    tmp_response = tmp_response.json()
    if "request_id" in tmp_response:
      req_id = tmp_response["request_id"]
      get_score_payload = {
      "request_id": f"{req_id}"
      }
    else:
      raise ValueError("Error in the Scoring")
    while True :
      sleep(0.05)
      scores_response = requests.request("POST", BASE_URL+"getscores", headers=headers, data=json.dumps(get_score_payload))
      scores_response = scores_response.json() 
      if scores_response['completed']:
          break
    scores = scores_response['data']
    clsss = scores['CLSSS']
    comet = [scores['COMET_QE, cased, punctuated'], scores['COMET_QE, cased, not punctuated'],  scores['COMET_QE, uncased, punctuated'], scores['COMET_QE, uncased, not punctuated']]
    comet = [float(_) for _ in comet]
    return comet + [clsss]


            

In [ ]:
df_2020 = pd.read_csv("./wmt-mqm-human-evaluation/newstest2020/ende/mqm_newstest2020_ende.tsv", sep="\t")
df_2020

In [ ]:
fe = FeatureExtractor()

In [ ]:
data = []
for source_text, subset in df_2020.groupby("source"):
    source_features = fe([source_text], "en")
    for i,row in tqdm(subset.iterrows(), total = subset.shape[0]):
        sample_id = format(i, '08d')
        if not os.path.exists(f"./data/{sample_id}.pkl"):
            target_text = clean_target(row.target)
            target_features = fe([target_text], "de")
            sample = {
                    "id": i,
                    "source_features": source_features,
                    "target_features": target_features,
                    "target-source": (target_features - source_features),
                    "category": row.category,
                    "severity": row.severity,
                    "commet_qe_clsss": get_scores(source_text, target_text),
                }
            
            with open(f"./data/{sample_id}.pkl", "wb") as tf:
                pickle.dump(sample,tf)